## 필요 라이브러리 설치

!pip install BeautifulSoup4

!pip install pandas

!pip install numpy

## 기본 html 파싱

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
res = requests.get('https://musclewiki.com/Directory')
soup = BeautifulSoup(res.content, 'html.parser')

lst = soup.find_all('tr')

## 필요 데이터 추출

In [3]:
muscle = []
exercise = []
equipment = []
difficulty = []
link = []

for item in lst:
    txt = [tmp for tmp in item.get_text().split('\n') if tmp]
    if 'Exercises' in item.get_text():
        muscle.append(txt[0].strip()[:-10])
        exercise.append([])
        equipment.append([])
        difficulty.append([])
        link.append([])
        
    else:
        exercise[-1].append(txt[0].strip())
        equipment[-1].append(txt[-2].strip())
        difficulty[-1].append(txt[-1].strip())
        tmp = 'https://musclewiki.com' + item.find('a')['href']
        link[-1].append(tmp)

## 데이터프레임 생성

In [4]:
df_lst = []

for i in range(len(muscle)):
    tmp_df = pd.DataFrame([exercise[i], equipment[i], difficulty[i], link[i]], 
                          index = ['exercise', 'equipment', 'difficulty', 'link']).transpose()
    df_lst.append(tmp_df)

## https://www.musclewiki.com + 이미지주소 하면 움짤 주소임!

df = df_lst[0]
a = df.loc[0, 'link']
b = requests.get(a)
s = BeautifulSoup(b.content, 'html.parser')
s.find(class_ = 'exercise-images-grid').find('img')['src']

a = df_lst[4]
r = requests.get(a.loc[0, 'link'])
s = BeautifulSoup(r.content, 'html.parser')
s.get_text()

for df in df_lst:
    for i in range(len(df)):
        tmp_link = df.loc[i, 'link']
        tmp_res = requests.get(tmp_link)
        tmp_soup = BeautifulSoup(tmp_res.content, 'html.parser')
        
        
        if '404' in tmp_soup.get_text():
            df.loc[i, 'img_link'] = '이미지 주소 오류'
        else:

            images = tmp_soup.find(class_ = 'exercise-images-grid').find_all('img')
            print(i, len(images))
    print()

In [25]:
a = df_lst[0].loc[0, 'link']
r = requests.get(a)
s = BeautifulSoup(r.content, 'html.parser')

i = s.find(class_ = 'exercise-images-grid')
i.find('a')['href']
b = s.find(class_ = 'steps-list')
t = [i for i in b.get_text().split('\n') if i]
g = ""
for i in range(len(t)):
    g = g + str(i + 1) + '. ' + t[i] + '\n'
    
print(g)
    
tmp = df_lst[0]
tmp.loc[0, 'step'] = g
tmp.loc[0, 'step']

1. While holding the upper arms stationary, curl the weights forward while contracting the biceps as you breathe out.
2. Continue the movement until your biceps are fully contracted and the bar is at shoulder level.
3. Hold the contracted position for a second and squeeze the biceps hard.
4. Slowly bring the weight back down to the starting position.



'1. While holding the upper arms stationary, curl the weights forward while contracting the biceps as you breathe out.\n2. Continue the movement until your biceps are fully contracted and the bar is at shoulder level.\n3. Hold the contracted position for a second and squeeze the biceps hard.\n4. Slowly bring the weight back down to the starting position.\n'

In [42]:
for df in df_lst:
    for i in range(len(df)):
        link = df.loc[i, 'link']
        tmp_res = requests.get(link)
        tmp_soup = BeautifulSoup(tmp_res.content, 'html.parser')
        
        if '404' in tmp_soup.get_text():
            df.loc[i, 'img_link'] = '이미지 주소 오류'
        else:
            image = tmp_soup.find(class_ = 'exercise-images-grid')
            df.loc[i, 'img_link'] = 'https://www.musclewiki.com' + image.find('img')['src']
            df.loc[i, 'video_link'] = image.find('a')['href']
            
            step_parse = tmp_soup.find(class_ = 'steps-list')
            step_lst = [item for item in step_parse.get_text().split('\n') if item]
            step = ""
            for j in range(len(step_lst)):
                step = step + str(j + 1) + '. ' + step_lst[j] + "\n"
            df.loc[i, 'step'] = step

df = df_lst[14]
len(df.loc[df['difficulty'] == 'Beginner']) + len(df.loc[df['difficulty'] != 'Beginner'])

df_lst[0].iloc[0]['link']
df_lst[0].loc[0, 'img_front'] = 'hello'
df_lst[0].loc[0, 'img_side'] = 'world'
df_lst[0]

In [46]:
df_lst[1].loc[10, 'step']

'1. Raise your arms to shoulder height, fully extended in front of you.\n2. Slowly bring your arms behind your back still at shoulder height.\n3. Pause for a few seconds and then return to starting position.\n'

In [47]:
beginner = []
master = []

for df in df_lst:
    beginner.append(df.loc[df['difficulty'] == 'Beginner'])
    master.append(df.loc[df['difficulty'] != 'Beginner'])

In [49]:
for i in range(len(df)):
    b_filename = "eng_Beginner_" + muscle[i]
    m_filename = "eng_Master_"+ muscle[i]
    
    b_df = beginner[i].loc[beginner[i]['difficulty'] == 'Beginner']
    m_df = master[i].loc[master[i]['difficulty'] != 'Beginner']
    
    b_df.to_csv('data/' + b_filename + '.csv', index = False)
    m_df.to_csv('data/' + m_filename + '.csv', index = False)

In [50]:
beginner[0]

,exercise,equipment,difficulty,link,step,img_link,video_link
0,Barbell Curl,Barbell,Beginner,https://musclewiki.com/Barbell/Male/Biceps/Bar...,"1. While holding the upper arms stationary, cu...",https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/0mylYdVWBf0
1,Dumbbell Curl,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Biceps/D...,1. Stand up straight with a dumbbell in each h...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/EU4HlSIpj5Y
2,Dumbbell Hammer Curl,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Biceps/D...,1. Hold the dumbbells with a neutral grip (thu...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/Dh3_gou_3TE
3,Goblet Curl,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Biceps...,1. Stand up straight with a kettlebell in both...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/oYmItLMNFg0
6,Single Arm Curl,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Biceps...,1. Stand up straight with a kettlebell in one ...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/jiheWHzzIJ4
7,Biceps Stretch Variation Five,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,1. Stand upright with your feet shoulder width...,https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=7jXK9pNJdJg%7C
8,Biceps Stretch Variation Four,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,1. Stand an arms width away from the wall.\n2....,https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=IRxy-WjmBRs%7C
9,Biceps Stretch Variation Three,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,"1. Bend your arm at the elbow, raising your ha...",https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=koCz9-ZVjRg%7C
10,Biceps Stretch Variation Two,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,"1. Stand facing the wall, an arm's width away....",https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=_qFHL-DoHG8%7C
11,Biceps Stretch Variation One,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,1. Stand one foot in front of the other with t...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/KPMttUk2xM8


## 난이도로 구분
- 추후 그룹화를 통해 기구 구분하여 각각 저장

In [30]:
beginner_lst = []
senior_lst = []

